<span style="color: Gold"> Extractive QA </span> : 책에서 문제의 답을 찾는다
- `start-logits` : 각 단어가 답의 시작인 확률 계산
- `end-logits` : 각 단어가 답의 끝일 확률 계산
- `argmaxV` : 가장 확률이 높은 구간 선택

In [1]:
# Extractive QA 시각화
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import matplotlib.pyplot as plt
import numpy as np

In [5]:
MODEL_NAME = 'distilbert-base-cased-distilled-squad'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\SAMSUNG\miniconda3\envs\llm_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SAMSUNG\.cache\huggingface\hub\models--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [7]:
question = "What is the capital of France?"
context = "Paris is the capital and largest city of France. The city has a population of 2.1 million."

In [8]:
# 토큰나이제이션

inputs = tokenizer(question, context, return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 1327, 1110, 1103, 2364, 1104, 1699,  136,  102, 2123, 1110, 1103,
         2364, 1105, 2026, 1331, 1104, 1699,  119, 1109, 1331, 1144,  170, 1416,
         1104,  123,  119,  122, 1550,  119,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [9]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] What is the capital of France? [SEP] Paris is the capital and largest city of France. The city has a population of 2. 1 million. [SEP]'

In [12]:
# 토큰리스트
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
tokens

['[CLS]',
 'What',
 'is',
 'the',
 'capital',
 'of',
 'France',
 '?',
 '[SEP]',
 'Paris',
 'is',
 'the',
 'capital',
 'and',
 'largest',
 'city',
 'of',
 'France',
 '.',
 'The',
 'city',
 'has',
 'a',
 'population',
 'of',
 '2',
 '.',
 '1',
 'million',
 '.',
 '[SEP]']

In [13]:
# 예측
with torch.no_grad():
  outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.0939, -3.7486, -6.0080, -5.7086, -5.8702, -6.6693, -4.2987, -3.1756,
         -4.0037,  9.6050, -2.3164, -0.6722, -0.0465, -4.4179, -1.2335, -2.8592,
         -4.7219,  0.3473, -2.1641, -0.4413, -2.7191, -5.4559, -4.6687, -4.0404,
         -6.9446, -2.4015, -6.9144, -4.8101, -4.1148, -4.0998, -4.0035]]), end_logits=tensor([[-1.6628, -3.1275, -5.5114, -6.3964, -5.4165, -6.5068, -3.2359, -3.9798,
         -3.7454,  9.3340, -0.6415, -2.9119,  0.9548, -4.1359, -1.7446, -0.0305,
         -4.1837,  3.6947,  1.4507, -3.4107, -3.0052, -5.5297, -5.5761, -3.7925,
         -6.3793, -3.9835, -5.3027, -4.0287, -0.2574, -1.0192, -3.7452]]), hidden_states=None, attentions=None)

In [16]:
# 로짓을 확률로 변환
start_prob = torch.softmax(outputs.start_logits, dim=-1)[0].numpy() # dim은 torch.Size에서 인덱스 1 또는 -1에 값이 오도록 한다
print(outputs.start_logits.shape)
end_prob = torch.softmax(outputs.end_logits, dim=-1)[0].numpy()

# 답변추출
answer_start = np.argmax(start_prob)
answer_end = np.argmax(end_prob) + 1
answer_tokens = tokens[answer_start:answer_end]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f'모델의 답변 : {answer}')
print(f'시작위치 : {answer_start} 토큰 {tokens[answer_start]}')
print(f'종료위치 : {answer_end} 토큰 {tokens[answer_end-1]}')
print(f'신뢰도 : {start_prob[answer_start] * end_prob[answer_end-1]}')

torch.Size([1, 31])
모델의 답변 : Paris
시작위치 : 9 토큰 Paris
종료위치 : 10 토큰 Paris
신뢰도 : 0.9952554702758789


In [ ]:
# 상위 5개 후보 출력
# 시작위치 상위 5개
top_start = np.argsort(-start_prob)[:5]
for i,index in enumerate(top_start,1):
  print(f'시작위치 : {i} 토큰 {tokens[index]} 확률: {start_prob[index]}')
# 종료위치 상위 5개
top_end =  np.argsort(-end_prob)[:5]
for i,index in enumerate(top_end,1):
  print(f'종료위치 : {i} 토큰 {tokens[index]} 확률: {end_prob[index]}')

  # 결론 : 확률이 고르게 분산 -> 답을 못찾은 경우, 특정 구간에 집중되면 확신있는 답을 찾음

시작위치 : 1 토큰 Paris 확률: 0.9996962547302246
시작위치 : 2 토큰 France 확률: 9.534510172670707e-05
시작위치 : 3 토큰 capital 확률: 6.430923531297594e-05
시작위치 : 4 토큰 The 확률: 4.333175820647739e-05
시작위치 : 5 토큰 the 확률: 3.4399869036860764e-05
종료위치 : 1 토큰 Paris 확률: 0.9955578446388245
종료위치 : 2 토큰 France 확률: 0.003539448603987694
종료위치 : 3 토큰 . 확률: 0.00037531490670517087
종료위치 : 4 토큰 capital 확률: 0.00022857278236187994
종료위치 : 5 토큰 city 확률: 8.533227082807571e-05


---

<span style="color: Gold"> offset mattping
- 토큰단위로 찾다보면 단어가 분리되어서 자연스럽지 못하기 때문에 문자위치로 변환하는 과정을 거쳐야 자연스러운 문장이 나온다

In [17]:
'''
문맥 : Paris is the capital of France.
        012345678901234 <- 문자 인덱스
토큰 : ['Paris', 'is', 'the', 'capital', 'of', 'France','.']
          0         1     2     3         <- 토큰인덱스
'''
#모델은 토큰인덱스로 작동(3번 토큰'capital')
#답변위치는 문자인덱스(13번째 문자부터 'capital'시작)
offset_mapping = [
    (0,5), # 토큰0: 'paris'는 문자 0~5 
    (6,8), # 토큰1: 'is'는 문자 6~8
    (9,12), 
    (13,20)
]
# 활용
# 문자위치 - 토큰위치 : Fine-tuning시 답변 레이블 생성
# 토큰위치 - 문자위치 : 예측 결과를 원문에 매칭

In [ ]:
# 토큰위치 <-> 문자위치 변환
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
question = "What is AI?"
context = "Artificial Intelligence (AI) is the simulation of human intelligence in machines."
inputs = tokenizer(
    question,
    context,
    return_tensors = 'pt',
    return_offsets_mapping = True
)
3# 토큰리스트 
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(tokens)
offset_mapping = inputs['offset_mapping'][0].tolist()
# 전체 텍스트(질문+문맥)
full_text = question + context
# 토큰별로 위치를 매핑
print(f"{'인덱스': <8}{'토큰':<25}{'문자위치':<18}{'원문매칭'}")
for idx, (token, (start, end)) in enumerate(zip(tokens, offset_mapping)):
        if start == end == 0:  # 특수 토큰
            matched_text = "[특수 토큰]"
            position = "N/A"
        else:
            # 원래 텍스트에서 추출
            matched_text = full_text[start:end]
            position = f"({start:3d}, {end:3d})"
        
        # 답변 영역 하이라이트
        highlight = "" if "AI" in token or "Intelligence" in token else "  "
        print(f"{idx:<8} {token:<25} {position:<18} '{matched_text}' {highlight}")

['[CLS]', 'What', 'is', 'AI', '?', '[SEP]', 'Art', '##ific', '##ial', 'Intelligence', '(', 'AI', ')', 'is', 'the', 'simulation', 'of', 'human', 'intelligence', 'in', 'machines', '.', '[SEP]']
인덱스     토큰                       문자위치              원문매칭
0        [CLS]                     N/A                '[특수 토큰]'   
1        What                      (  0,   4)         'What'   
2        is                        (  5,   7)         'is'   
3        AI                        (  8,  10)         'AI' 
4        ?                         ( 10,  11)         '?'   
5        [SEP]                     N/A                '[특수 토큰]'   
6        Art                       (  0,   3)         'Wha'   
7        ##ific                    (  3,   7)         't is'   
8        ##ial                     (  7,  10)         ' AI'   
9        Intelligence              ( 11,  23)         'Artificial I' 
10       (                         ( 24,  25)         't'   
11       AI                        ( 25,  27)     

In [19]:
# 실제 답변 위치
# 예를 들어, 답변이 하기와 같다면, 
answer = "Artificial Intelligence"
# 문맥에서 답변의 문자 위치 찾기
answer_start_char = context.index(answer)
answer_end_char = answer_start_char + len(answer)

# 문자위치 --> 토큰위치로 변환
answer_start_token = None
answer_end_token = None

# 질문부분 건너뛰기 (sequence_idf)
sequence_id = inputs.sequence_ids(0)# token_type_ids 를 대체하는게 sequence_ids 라 보면 됨
context_start_idx = sequence_id.index(1)# 첫번째 문맥의 시작

# 문맥에서 절대 위치를 계산하기 위해 질문 길이 보정
question_length = len(question) + 1 # +1은 공백을 의미
abs_answer_start = answer_start_char + question_length
abs_answer_end = answer_end_char + question_length
print(f"질문 길이 보정 + (question_length)")
print(f"절대문자위치 : {abs_answer_start} - {abs_answer_end}")

질문 길이 보정 + (question_length)
절대문자위치 : 12 - 35


In [22]:
for idx in range(context_start_idx, len(offset_mapping)):
  start, end = offset_mapping[idx]
  if start == end == 0:  # 특수 토큰 무시
      continue
  # 답변 시작 토큰 찾기
  if answer_start_token is None and end > abs_answer_start:
      answer_start_token = idx
      print(f"    시작 토큰 발견!")
      print(f"     인덱스: {idx}")
      print(f"     토큰: '{tokens[idx]}'")
      print(f"     오프셋: ({start}, {end})")
      print()

  # 답변 종료 토큰 찾기
  if answer_end_token is None and start >= abs_answer_end:
      answer_end_token = idx - 1
      print(f"    종료 토큰 발견!")
      print(f"      인덱스: {idx - 1}")
      print(f"      토큰: '{tokens[idx-1]}'")
      print(f"      오프셋: {offset_mapping[idx-1]}")
      break

  print("\n" + "="*80)
  print("최종 결과")
  print("="*80)

  if answer_start_token and answer_end_token:
    print(f"\n 변환 성공!")
    print(f"   토큰 위치: {answer_start_token} ~ {answer_end_token}")
    print(f"   토큰 리스트: {tokens[answer_start_token:answer_end_token+1]}")

    # 역변환 확인
    reconstructed = tokenizer.convert_tokens_to_string(
        tokens[answer_start_token:answer_end_token+1]
    )
    print(f"   복원된 텍스트: '{reconstructed}'")
    print(f"   원본 텍스트: '{answer}'")

    if answer.lower() in reconstructed.lower():
        print(f"\n   검증 성공! 완벽하게 매칭됩니다.")
    else:
        print(f"\n   부분 일치 (토크나이제이션으로 인한 차이)")
  else:
   print("\n 변환 실패 - 답변을 찾을 수 없습니다")


최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 위치: 18 ~ 19
   토큰 리스트: ['intelligence', 'in']
   복원된 텍스트: 'intelligence in'
   원본 텍스트: 'Artificial Intelligence'

   부분 일치 (토크나이제이션으로 인한 차이)

최종 결과

 변환 성공!
   토큰 

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# SQuAD 데이터 전처리
# SQuAD 모델이 이해할 수 있는 형식으로 바꾸는 과정에 대한 이해

sample = {
    "question": "When was the Eiffel Tower built?",
    "context": "The Eiffel Tower was built in 1889 for the Paris World's Fair. It was designed by Gustave Eiffel and remains one of the most iconic structures in the world.",
    "answers": {
        "text": ["1889"],
        "answer_start": [31]  # "1889"의 문자 시작 위치
    }
}
# 1. 토크나이제이션
#[CLS] When was ...[SEP] the eiffel tower ... [SEP]
inputs = tokenizer (
  sample['question'],
  sample['context'],
  max_length = 384,
  truncation = 'only_second', # 문맥만 자르기
  return_offsets_mapping = True,  # 오프셋 매핑
  return_tensors = 'pt',
  padding = 'max_length'  # 최대 길이로 패딩
)
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(f'총 토큰 수 : {len(tokens)}')
print(f'실제 : {inputs['attention_mask'][0].tolist().count(1)}')
print(f'패딩 : {inputs['attention_mask'][0].tolist().count(0)}')

# 2. 답변 문자 위치 확인
answer_text = sample['answers']['text'][0]
answer_start_char = sample['answers']['answer_start'][0]
answer_end_char = answer_start_char + len(answer_text)
print(f'답변 텍스트 : {answer_text}')
print(f'답변 시작 : {answer_start_char}')
print(f'답변 종료 : {answer_end_char}')
print(f'검증 : {sample["context"][answer_start_char:answer_end_char]}')

# 3. sequence_ids로 문맥 범위 찾기
# context_range = [10,45] # 토큰10번~45번이 문맥
# token_type_id와 유사 / 문장의 종류 0:질문 1:문장 None:특수토큰 또는 패딩

sequence_ids = inputs.sequence_ids(0)
# 문맥의 시작과 끝
context_start = sequence_ids.index(1)
context_end = len(sequence_ids) - sequence_ids[::-1].index(1)-1
print(f'특수토큰 : {sequence_ids.count(None)}')
print(f'0 질문: {sequence_ids.count(0)}')
print(f'1 질문: {sequence_ids.count(1)}')
print(f'문맥 시작 : {context_start}')
print(f'문맥 종료 : {context_end}')
print(f'문맥 첫번째 토큰 : {tokens[context_start]}')
print(f'문맥 마지막 토큰 : {tokens[context_end]}')


"\n{\n  'input_ids' : ...\n  'attention_mask' : ...\n  'token_type_ids' : ...\n  'start_positions' : 18\n  'end_positions' : 19\n}\n"